In [21]:
#Importing Libraries

import pandas as pd
from sklearn import metrics
from sklearn.preprocessing import scale
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [22]:
### Data preprocessing
train = pd.read_csv("D:/Fall/APM/project/data_train_pre.csv", )
test = pd.read_csv("D:/Fall/APM/project/data_test_pre.csv", )

train[train['scheme_management']=="None"]['scheme_management'] = "unknown"
test[test['scheme_management']=="None"]['scheme_management'] = "unknown"
train=train.drop('extraction_type',axis=1,inplace=False)

test=test.drop('extraction_type',axis=1,inplace=False)
train=train.drop(['Unnamed: 0','id','date_recorded','quality_group','quantity_group', 'management_group','source_type','management_group','extraction_type_group',
                  'district_code','region_code','payment_type','scheme_name'],axis=1,inplace=False)
test=test.drop(['Unnamed: 0','date_recorded','quality_group','quantity_group', 'management_group','source_type','management_group','extraction_type_group',
                  'district_code','region_code','payment_type','scheme_name'],axis=1,inplace=False)
count=train['status_group'].value_counts()

count

train.columns.values

array(['region', 'amount_tsh', 'funder', 'gps_height', 'installer',
       'longitude', 'latitude', 'wpt_name', 'num_private', 'basin', 'lga',
       'population', 'public_meeting', 'scheme_management', 'permit',
       'extraction_type_class', 'management', 'payment', 'water_quality',
       'quantity', 'source', 'source_class', 'waterpoint_type',
       'waterpoint_type_group', 'age', 'loc_type', 'employment_rate',
       'popdeath_rate', 'status_group', 'crime_rating'], dtype=object)

In [23]:
# creating train and hold out data set
import random
length=len(train)
rows = random.sample(train.index, length/2)
train_s1 = train.ix[rows]
count=train_s1['status_group'].value_counts()
train_s2 = train.drop(rows)

count


functional                 16068
non functional             11485
functional needs repair     2147
dtype: int64

In [4]:
# writing the downsampling fucntion
def dnsmpl(xtrain,count,p):
    train=xtrain
    positive=count['functional needs repair']
    negative=int(positive*((1-p)/p))
    train_0=train[train['status_group']<>'functional needs repair']
    train_1=train[train['status_group']=='functional needs repair']
    train_0_down_sampled = train_0.sample( negative)
    train_final=train_1.append(train_0_down_sampled)
    return(train_final)

y=train_s2['status_group']
train1=train_s1.drop('status_group',axis=1,inplace=False)
train_s2_dum=pd.get_dummies(train_s2)
y_binary=pd.get_dummies(y)
y_bin=y_binary['functional needs repair']

In [54]:
# finding the best downsampling value
f1_score=[]  
for p in xrange(10,60,10):
    p1=float(p)/float(100)
    xtrain=dnsmpl(train_s1,count,p1)
    y1=xtrain['status_group']
    xtrain=xtrain.drop('status_group',axis=1,inplace=False)
    model = RandomForestClassifier(n_estimators=200)
    xtrain_dum=pd.get_dummies(xtrain)
    result = model.fit(xtrain_dum, y1)
    columns=train_s2_dum.columns.values
    columns1=xtrain_dum.columns.values
    columns2=set(columns1).intersection(columns)
    prediction_train = model.predict(train_s2_dum[list(columns2)])
    prediction_train_dummy=pd.get_dummies(prediction_train)
    prediction_train_dum=1-prediction_train_dummy[['functional','non functional']].max(axis=1)
    f1=metrics.f1_score(y_bin, prediction_train_dum)
    print 'f1',metrics.precision_recall_fscore_support(y_bin, prediction_train_dum),'accuracy',metrics.accuracy_score(y, prediction_train)
    f1_score.append(f1)
lr_results1=pd.Series(f1_score) 
lr_results_best=lr_results1.order(ascending=False).index[0]
print 'Best parameter for under sampling', lr_results_best 
lr_results1.order(ascending=False)

f1 (array([ 0.92808081,  0.        ]), array([ 1.,  0.]), array([ 0.96269908,  0.        ]), array([27564,  2136], dtype=int64)) accuracy 0.391582491582
f1 (array([ 0.9280687,  0.       ]), array([ 0.9998186,  0.       ]), array([ 0.9626085,  0.       ]), array([27564,  2136], dtype=int64)) accuracy 0.471515151515
f1 (array([ 0.92770307,  0.02777778]), array([ 0.99111159,  0.00327715]), array([ 0.95835964,  0.00586265]), array([27564,  2136], dtype=int64)) accuracy 0.547306397306
f1 (array([ 0.92189349,  0.06871166]), array([ 0.33913801,  0.62921348]), array([ 0.49586251,  0.12389381]), array([27564,  2136], dtype=int64)) accuracy 0.22404040404
f1 (array([ 0.90228359,  0.07017292]), array([ 0.06163837,  0.91385768]), array([ 0.11539376,  0.13033753]), array([27564,  2136], dtype=int64)) accuracy 0.0963636363636
Best parameter for under sampling 4


4    0.130338
3    0.123894
2    0.005863
1    0.000000
0    0.000000
dtype: float64

# Random Forest

In [12]:
#finding the optimal trees
f1_score=[]  
for p in xrange(100,600,100):
    #p1=float(p)/float(100)
    xtrain=train_s1
    y1=xtrain['status_group']
    xtrain=xtrain.drop('status_group',axis=1,inplace=False)
    model = RandomForestClassifier(n_estimators=p)
    xtrain_dum=pd.get_dummies(xtrain)
    columns=train_s2_dum.columns.values
    columns1=xtrain_dum.columns.values
    columns2=set(columns).intersection(columns1)
    result = model.fit(xtrain_dum[list(columns2)], y1)
    prediction_train = model.predict(train_s2_dum[list(columns2)])
    #prediction_train_dummy=pd.get_dummies(prediction_train)
    #prediction_train_dum=1-prediction_train_dummy[['functional','non functional']].max(axis=1)
    f1=metrics.f1_score(y, prediction_train)
    print 'f1',metrics.f1_score(y, prediction_train),'accuracy',metrics.accuracy_score(y, prediction_train)
    f1_score.append(f1)
lr_results1=pd.Series(f1_score) 
lr_results_best=lr_results1.order(ascending=False).index[0]
print 'Best parameter for under sampling', lr_results_best 
lr_results1.order(ascending=False)

f1 0.78826518541 accuracy 0.794175084175
f1 0.787674782482 accuracy 0.793535353535
f1 0.790208408984 accuracy 0.795858585859
f1 0.789447123152 accuracy 0.795218855219
f1 0.78883953071 accuracy 0.794511784512
Best parameter for under sampling 2


2    0.790208
3    0.789447
4    0.788840
0    0.788265
1    0.787675
dtype: float64

In [18]:
#Finding the optimal parameters
c=['auto','log2',None]
for p in c:
    #p1=float(p)/float(100)
    xtrain=train_s1
    y1=xtrain['status_group']
    xtrain=xtrain.drop('status_group',axis=1,inplace=False)
    model = RandomForestClassifier(max_features=p,n_estimators=300 )
    xtrain_dum=pd.get_dummies(xtrain)
    columns=train_s2_dum.columns.values
    columns1=xtrain_dum.columns.values
    columns2=set(columns).intersection(columns1)
    result = model.fit(xtrain_dum[list(columns2)], y1)
    prediction_train = model.predict(train_s2_dum[list(columns2)])
    #prediction_train_dummy=pd.get_dummies(prediction_train)
    #prediction_train_dum=1-prediction_train_dummy[['functional','non functional']].max(axis=1)
    f1=metrics.f1_score(y, prediction_train)
    print 'f1',metrics.f1_score(y, prediction_train),'accuracy',metrics.accuracy_score(y, prediction_train)
    f1_score.append(f1)
lr_results1=pd.Series(f1_score) 
lr_results_best=lr_results1.order(ascending=False).index[0]
print 'Best parameter for under sampling', lr_results_best 
lr_results1.order(ascending=False)



f1 0.788908301282 accuracy 0.794781144781
f1 0.787675493799 accuracy 0.793367003367
f1 0.787631524432 accuracy 0.793400673401
Best parameter for under sampling 2


2    0.790208
3    0.789447
5    0.788908
4    0.788840
0    0.788265
6    0.787675
1    0.787675
7    0.787632
dtype: float64

In [ ]:
###random forest with PCA
train_dum=pd.get_dummies(train1)
test_dum=pd.get_dummies(test)
columns=test_dum.columns.values
columns1=train_dum.columns.values
columns2=set(columns).intersection(columns1)
train_dum=train_dum[list(columns2)]
test_dum=test_dum[list(columns2)]
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
print len(columns2)
train_dum_scale=scale(train_dum)
test_dum_scale=scale(test_dum)
pca = PCA(n_components=100)
pca_fit=pca.fit(train_dum_scale)
train_pca=pca_fit.transform(train_dum_scale)
test_pca=pca_fit.transform(test_dum_scale)
pca.explained_variance_ratio_ 
train_pca=np.matrix(train_pca)
train_pca=pd.DataFrame(train_pca)
test_pca=np.matrix(test_pca)
test_pca=pd.DataFrame(test_pca)
model = RandomForestClassifier(n_estimators=1000,oob_score=True)
result = model.fit(train_dum, y)
out_of_bag_prediction_for_x = model.oob_score_
print(out_of_bag_prediction_for_x)

prediction_train = model.predict(test_dum[list(columns2)])
print test.columns.values
test['status_group']=prediction_train
results=test[['id','status_group']]
results.to_csv('E:/results_rf_1000.csv')


# Gradient Boosting Classifier

In [20]:
from sklearn.ensemble import GradientBoostingClassifier
f1_score=[]  
for p in xrange(100,600,100):
    #p1=float(p)/float(100)
    xtrain=train_s1
    y1=xtrain['status_group']
    xtrain=xtrain.drop('status_group',axis=1,inplace=False)
    model = GradientBoostingClassifier(n_estimators=p, learning_rate=1.0, random_state=0)
    xtrain_dum=pd.get_dummies(xtrain)
    columns=train_s2_dum.columns.values
    columns1=xtrain_dum.columns.values
    columns2=set(columns).intersection(columns1)
    result = model.fit(xtrain_dum[list(columns2)], y1)
    prediction_train = model.predict(train_s2_dum[list(columns2)])
    #prediction_train_dummy=pd.get_dummies(prediction_train)
    #prediction_train_dum=1-prediction_train_dummy[['functional','non functional']].max(axis=1)
    f1=metrics.f1_score(y, prediction_train)
    print 'f1',metrics.f1_score(y, prediction_train),'accuracy',metrics.accuracy_score(y, prediction_train)
    f1_score.append(f1)
lr_results1=pd.Series(f1_score) 
lr_results_best=lr_results1.order(ascending=False).index[0]
print 'Best parameter for under sampling', lr_results_best 
lr_results1.order(ascending=False)



f1 0.76361807938 accuracy 0.773333333333
f1 0.767487223639 accuracy 0.775151515152
f1 0.767505410009 accuracy 0.77468013468
f1 0.769702412826 accuracy 0.776464646465
f1 0.767372272309 accuracy 0.773939393939
Best parameter for under sampling 3


3    0.769702
2    0.767505
1    0.767487
4    0.767372
0    0.763618
dtype: float64

In [25]:
# finding the optimal depth of trees
from sklearn.ensemble import GradientBoostingClassifier
f1_score=[]  
for p in xrange(5,35,5):
    #p1=float(p)/float(100)
    xtrain=train_s1
    y1=xtrain['status_group']
    xtrain=xtrain.drop('status_group',axis=1,inplace=False)
    model = GradientBoostingClassifier(n_estimators=100,max_depth=p, learning_rate=1.0, random_state=0)
    xtrain_dum=pd.get_dummies(xtrain)
    columns=train_s2_dum.columns.values
    columns1=xtrain_dum.columns.values
    columns2=set(columns).intersection(columns1)
    result = model.fit(xtrain_dum[list(columns2)], y1)
    prediction_train = model.predict(train_s2_dum[list(columns2)])
    #prediction_train_dummy=pd.get_dummies(prediction_train)
    #prediction_train_dum=1-prediction_train_dummy[['functional','non functional']].max(axis=1)
    f1=metrics.f1_score(y, prediction_train)
    print 'f1',metrics.f1_score(y, prediction_train),'accuracy',metrics.accuracy_score(y, prediction_train)
    f1_score.append(f1)
lr_results1=pd.Series(f1_score) 
lr_results_best=lr_results1.order(ascending=False).index[0]
print 'Best parameter for under sampling', lr_results_best 
lr_results1.order(ascending=False)

f1 0.848596383147 accuracy 0.851616161616
f1 0.885869044629 accuracy 0.886936026936
f1 0.887151324178 accuracy 0.88835016835


KeyboardInterrupt: 

In [30]:
##checking on the training set in driven data org
test.dum=pd.get_dummies(test)

xtrain=train_s1
y1=xtrain['status_group']
xtrain=xtrain.drop('status_group',axis=1,inplace=False)
model = GradientBoostingClassifier(n_estimators=100,max_depth=15, learning_rate=1.0, random_state=0)
xtrain_dum=pd.get_dummies(xtrain)
columns=test.dum.columns.values
columns1=xtrain_dum.columns.values
columns2=set(columns).intersection(columns1)
result = model.fit(xtrain_dum[list(columns2)], y1)
prediction_train = model.predict(test.dum[list(columns2)])

In [33]:
prediction_train
test['status_group']=prediction_train
results=test[['status_group','id']]

results.to_csv('E:/results_gbm_100.csv')

